# Repeated measurements on Starmon-5

In [1]:
from typing import List
from tqdm import tqdm
import numpy as np
import itertools as it

import qiskit
import qiskit.tools.jupyter
from qiskit import QuantumCircuit
import qiskit.circuit.library.standard_gates as gates


from importlib import reload
import inspire_experiments
reload(inspire_experiments)
from inspire_experiments import ExperimentData, get_starmon_status, inspire_login, get_file_header, measure_readout_correction

# %matplotlib notebook
%matplotlib inline
%qiskit_version_table

In [2]:
api, backend = inspire_login()
backend.status()

Backend status:  EXECUTION


In [11]:
get_starmon_status(api)

'EXECUTION'

## Entanglement by msmt


In [9]:
def measure_entanglement_by_msmt(
        qubits: List[int],
        tomo_qubits: List[int]=[],
        sim_msmt: bool=True,
        exp_basename: str="entanglement_by_msmt",
        extra_label: str=None,
        show_circuit: bool=False,
        send_jobs: bool=True
):
    jobs = []
    circuits = []
    if tomo_qubits:
        tomo_gates = {'X': gates.RYGate(-np.pi/2), 'Y': gates.RXGate(np.pi/2), 'Z': gates.IGate()}
        tomo_combinations = list(it.product(tomo_gates.keys(), repeat=len(tomo_qubits)))
    else:
        tomo_combinations = ['Z']

    for tomo_option in tqdm(tomo_combinations, total=len(tomo_combinations)):
        circuit = QuantumCircuit(5,5)
        # circuit.rx(np.pi/2, qubits[1])
        # circuit.ry(-np.pi/2, qubits[0])
        # circuit.ry(-np.pi/2, qubits[2])

        for qb in qubits:
            circuit.ry(np.pi/2, qb)

        circuit.barrier(range(5))
        circuit.cz(qubits[1], qubits[0])
        circuit.id(qubits[1])
        # circuit.id(qubits[1])
        circuit.cz(qubits[1], qubits[2])
        circuit.barrier(range(5))

        for qb in qubits:
            circuit.ry(-np.pi/2, qb)

        # circuit.rx(np.pi/2, qubits[1])
        # circuit.ry(np.pi/2, qubits[0])
        # circuit.ry(np.pi/2, qubits[2])
        circuit.barrier(range(5))
        if qubits[1] in tomo_qubits:
            idx = tomo_qubits.index(qubits[1])
            circuit.append(tomo_gates[tomo_option[idx]], [qubits[1]])
        elif sim_msmt:
            circuit.id(qubits[1])
        circuit.measure(qubits[1], qubits[1])


        # for qb in set(qubits).intersection(tomo_qubits):
        #     idx = tomo_qubits.index(qb)
        #     circuit.append(tomo_gates[tomo_option[idx]], [qb])
        #     circuit.measure(qb, qb)

        if not sim_msmt:
            # add waiting time via ID gates on data qubits
            for _ in range(100):
                circuit.id(qubits[0])
                circuit.id(qubits[-1])
            circuit.barrier(range(5))

        # for qb in set(qubits).difference(tomo_qubits):
        #     circuit.id(qb)
        #     circuit.measure(qb, qb)

        if qubits[0] in tomo_qubits:
            idx = tomo_qubits.index(qubits[0])
            circuit.append(tomo_gates[tomo_option[idx]], [qubits[0]])
        elif sim_msmt:
            circuit.id(qubits[0])
        circuit.measure(qubits[0], qubits[0])

        if qubits[2] in tomo_qubits:
            idx = tomo_qubits.index(qubits[2])
            circuit.append(tomo_gates[tomo_option[idx]], [qubits[2]])
        elif sim_msmt:
            circuit.id(qubits[2])
        circuit.measure(qubits[2], qubits[2])

        if show_circuit:
            display(circuit.draw(output='mpl', scale=1))
        circuits += [circuit]

        exp_name = exp_basename + f"_qbs{qubits}"
        exp_name = exp_name + "_sim-msmt" if sim_msmt else exp_name
        exp_name = exp_name + extra_label if extra_label else exp_name
        exp_name = exp_name + f"_tom{tomo_qubits}" + f"_M{tomo_option}" if tomo_qubits else exp_name
        print(exp_name)

        if send_jobs:
            header = get_file_header(circuit)
            print(get_starmon_status(api))
            job = qiskit.execute(circuit, shots=2**14, optimization_level=0, backend=backend)
            ExperimentData.save_job_result(job, exp_name, header)
            jobs += [job]

    return jobs, circuits


In [12]:
jobs, circuits = measure_entanglement_by_msmt([0,2,4], tomo_qubits=[0,4], send_jobs=True, sim_msmt=False, show_circuit=False)

  0%|          | 0/9 [00:00<?, ?it/s]

entanglement_by_msmt_qbs[0, 2, 4]_tom[0, 4]_M('X', 'X')
EXECUTION


17:10:28 - inspire_experiments.data_utils:INFO - Saving results for job 7164760 in ./data/
 11%|█         | 1/9 [01:20<10:45, 80.64s/it]

entanglement_by_msmt_qbs[0, 2, 4]_tom[0, 4]_M('X', 'Y')
EXECUTION


17:11:49 - inspire_experiments.data_utils:INFO - Saving results for job 7164761 in ./data/
 22%|██▏       | 2/9 [02:55<10:23, 89.04s/it]

...

17:18:18 - inspire_experiments.data_utils:INFO - Saving results for job 7164767 in ./data/
 78%|███████▊  | 7/9 [08:43<02:08, 64.21s/it]

entanglement_by_msmt_qbs[0, 2, 4]_tom[0, 4]_M('Z', 'Y')
IDLE


17:19:11 - inspire_experiments.data_utils:INFO - Saving results for job 7164768 in ./data/
 89%|████████▉ | 8/9 [09:36<01:00, 60.66s/it]

entanglement_by_msmt_qbs[0, 2, 4]_tom[0, 4]_M('Z', 'Z')
IDLE


17:20:04 - inspire_experiments.data_utils:INFO - Saving results for job 7164769 in ./data/
100%|██████████| 9/9 [10:29<00:00, 69.97s/it]


In [13]:
jobs2, circuits2 = measure_entanglement_by_msmt([0,2,4], tomo_qubits=[0,4], send_jobs=True, sim_msmt=True, show_circuit=False)

  0%|          | 0/9 [00:00<?, ?it/s]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_tom[0, 4]_M('X', 'X')
IDLE


17:20:58 - inspire_experiments.data_utils:INFO - Saving results for job 7164770 in ./data/
 11%|█         | 1/9 [00:47<06:20, 47.54s/it]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_tom[0, 4]_M('X', 'Y')
IDLE


17:21:45 - inspire_experiments.data_utils:INFO - Saving results for job 7164771 in ./data/
 22%|██▏       | 2/9 [01:38<05:45, 49.43s/it]

...

17:25:56 - inspire_experiments.data_utils:INFO - Saving results for job 7164776 in ./data/
 78%|███████▊  | 7/9 [05:48<01:40, 50.41s/it]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_tom[0, 4]_M('Z', 'Y')
EXECUTION


17:26:47 - inspire_experiments.data_utils:INFO - Saving results for job 7164777 in ./data/
 89%|████████▉ | 8/9 [06:35<00:49, 49.19s/it]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_tom[0, 4]_M('Z', 'Z')
IDLE


17:27:33 - inspire_experiments.data_utils:INFO - Saving results for job 7164778 in ./data/
100%|██████████| 9/9 [07:27<00:00, 49.78s/it]


In [14]:
measure_readout_correction(qubits=[0,4], backend=backend, send_jobs=True, show_circuit=False)

17:28:25 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[0, 4]_state00
17:28:25 - inspire_experiments.experiment_library:INFO - Measuring readout correction: state ('0', '0')
17:28:26 - inspire_experiments.data_utils:INFO - Saving results for job 7164779 in ./data/
17:29:16 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[0, 4]_state01
17:29:16 - inspire_experiments.experiment_library:INFO - Measuring readout correction: state ('0', '1')
17:29:17 - inspire_experiments.data_utils:INFO - Saving results for job 7164780 in ./data/
17:30:08 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[0, 4]_state10
17:30:08 - inspire_experiments.experiment_library:INFO - Measuring readout correction: state ('1', '0')
17:30:08 - inspire_experiments.data_utils:INFO - Saving results for job 7164781 in ./data/
17:30:59 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[0, 4]_state11
17:30:59 - inspire_experiments.experime

([<quantuminspire.qiskit.qi_job.QIJob at 0x7fc36028aca0>,
  <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fc360533760>])